In [ ]:
%load_ext autoreload
%autoreload 2

Description:

Ce notebook est utile pour afficher les courbes de tendance de SoH par modèle (pour l'instant tesla) et on peut même aller plus loin avec les courbe de tendance de SoH pour les versions (ou tesla code).

Il y'a la possibilité d'ajouter les intervals de confiance de ces courbes. 

In [ ]:
import pandas as pd
from core.sql_utils import *
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

from transform.insights_results.trendlines import *


In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT *  FROM  vehicle v	
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join vehicle_data vd 
            on vd.vehicle_id  = v.id
            join oem o 
            on o.id = vm.oem_id
            join battery b 
            on b.id = vm.battery_id""")
            #where o.id in ('5b89ebf7-8774-41ec-97f8-7dd4e8a2fd0f');""")
                             , con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)

In [ ]:
dbeaver_df

In [ ]:
dbeaver_df.columns

In [ ]:
# on drop MTY13/MT336/MT337
dbeaver_filtered = dbeaver_df[~dbeaver_df['version'].isin(['MT336','MT337'])][['version', 'soh', 'odometer', 'model_name', 'type', 'vin', 'oem_name']].copy().dropna()


In [ ]:
px.scatter(dbeaver_filtered[['version', 'soh', 'odometer']],  x='odometer', y='soh', color="version")

## Graphs des tendances 

In [ ]:
def trendline_apply(x, f):
    return eval(f)
    
    

In [ ]:
dbeaver_filtered

In [ ]:
dbeaver_filtered

In [ ]:
{"trendline": "1 + -0.07111948116198542 * np.log1p(x/64332.975567039895)"}	
{"trendline": "0.9498052237392416 + -0.055693865839002835 * np.log1p(x/ 95052.87170433612)"}	
{"trendline": "1.0119164014772302+ -0.3674772607185703 * np.log1p(x/626220.0892625029)"}

In [ ]:
{"trendline": "0.9533916940704387 + -405.07962452110735 * np.log1p(x/9221862704.769915)"}

In [ ]:

px.scatter(
    x=np.linspace(0, 200000, 1000),
    y=trendline_apply(np.linspace(0, 200000, 1000), "0.9533916940704387 + -405.07962452110735 * np.log1p(x/9221862704.769915)"),
    #mode='lines',
    #line=dict(color='red'),
    #name='Fit'
)

    # fig.add_trace(go.Scatter(
    #     x=x_sorted,
    #     y=trendline_apply(x_sorted, trendline_max['trendline']),
    #     mode='lines',
    #     line=dict(color='green'),
    #     name='Upper'
    # ))

    # fig.add_trace(go.Scatter(
    #     x=x_sorted,
    #     y=trendline_apply(x_sorted, trendline_min['trendline']),
    #     mode='lines',
    #     line=dict(color='green'),
    #     name='Lower'
    # ))


In [ ]:
import plotly.graph_objects as go

for marque in ["tesla", 'renault', "mercedes", "ford", "volvo"]:
    df_marque = dbeaver_filtered[dbeaver_filtered['oem_name'] == marque]
    trendline, trendline_max, trendline_min = get_trendlines(df_marque)
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_marque['odometer'],
        y=df_marque['soh'],
        mode='markers',
        marker_color='rgba(50, 182, 193, .9)',
        name='SoH compute'
    ))

    x_sorted = df_marque['odometer'].sort_values()
    print()
    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline['trendline']),
        mode='lines',
        line=dict(color='red'),
        name='Fit'
    ))

    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline_max['trendline']),
        mode='lines',
        line=dict(color='green'),
        name='Upper'
    ))

    fig.add_trace(go.Scatter(
        x=x_sorted,
        y=trendline_apply(x_sorted, trendline_min['trendline']),
        mode='lines',
        line=dict(color='green'),
        name='Lower'
    ))

    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title='Odometer',
        yaxis_title='State of Health (SoH)',
        legend_title='Légende',
        title=f"Marque: {marque}",
        template='plotly_white'
    )

    fig.show()

### GSheet

In [ ]:
import gspread
from google.oauth2.service_account import Credentials

# <!-- # Charger les identifiants
# creds = Credentials.from_service_account_file(
#     'chemin/vers/le/fichier.json',
#     scopes=["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"] -->
# )

In [ ]:
creds = Credentials.from_service_account_file(
    '../../ingestion/vehicle_info/config/config.json',
    scopes=["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
)

In [ ]:
client = gspread.authorize(creds)


In [ ]:
sheet = client.open("202505 - Courbes SoH")  


In [ ]:
worksheets = sheet.worksheets()


In [ ]:
worksheets

In [ ]:
coubes_sheet = sheet.worksheet("Courbes OS")


In [ ]:
df_spreed_sheet = pd.DataFrame(coubes_sheet.get_all_values()[1:], columns=coubes_sheet.get_all_values()[0])

In [ ]:
df_spreed_sheet["Odomètre (km)"] = df_spreed_sheet["Odomètre (km)"].apply(lambda x: x.replace(" ", "")).astype(int)
df_spreed_sheet["SoH"] = df_spreed_sheet["SoH"].apply(lambda x: x.replace("%", "")).astype(float)
df_spreed_sheet.rename(columns={"Odomètre (km)": "odometer",
                                "SoH": "soh"}, inplace=True)

In [ ]:
px.scatter(df_spreed_sheet, x='odometer', y='soh', color='OEM')

In [ ]:
from transform.insights_results.trendlines import *

In [ ]:
df_spreed_sheet.dtypes

In [ ]:
trendine_dict = {}
for oem in df_spreed_sheet.OEM.unique():
    if (oem in ['Renault', 'Dacia', 'Nissan', 'Peugeot']):
        print(oem)
        trendine_dict[oem] = {}
        x_data = df_spreed_sheet[df_spreed_sheet['OEM']==oem].dropna(subset=['soh', 'odometer'])['odometer'].values
        y_data = df_spreed_sheet[df_spreed_sheet['OEM']==oem].dropna(subset=['soh', 'odometer'])['soh'].values
        x_sorted = x_data[np.argsort(x_data)]
        y_sorted = y_data[np.argsort(x_data)]
        coef_, _ = curve_fit(log_function, x_sorted, y_sorted, p0=[-5, 10000], maxfev=10000)
        y_fit = log_function(x_sorted, *coef_)
        window_size = 1
        local_std = np.array([
            np.std(y_sorted[max(0, i-window_size):min(len(y_sorted), i+window_size)])
            for i in range(len(y_sorted))
        ])

        smooth = np.linspace(local_std[0], local_std[-1], len(local_std))

        y_lower = y_fit - 1 * smooth
        y_upper = y_fit +  1 * smooth
        def log_function_min(x, a, b):
                return y_lower.max() + a * np.log1p(x/b)
        def log_function_max(x, a, b):
            return y_upper.max() + a * np.log1p(x/b)
        coef_lower, _ = curve_fit(log_function_min, x_sorted, y_lower, maxfev=1000000)
        coef_upper, _ = curve_fit(log_function_max, x_sorted, y_upper, maxfev=1000000)

        trendline = {"trendline": f"100 + {coef_[0]} * np.log1p(x/{coef_[1]})"}
        trendline_max ={"trendline_max": f"{max(y_upper.max(), 1)}+ {coef_upper[0]} * np.log1p(x/{coef_upper[1]})"}
        trendline_min ={"trendline_min": f"{min(y_lower.max(), 1)} + {coef_lower[0]} * np.log1p(x/ {coef_lower[1]})"}
        trendine_dict[oem] = {}
        trendine_dict[oem] = [trendline, trendline_max, trendline_min]
        # trendine_dict['trendline_max'] = trendline_max
        # trendine_dict['trendline_min'] = trendline_min

In [ ]:
trendine_dict

In [ ]:
df_spreed_sheet[df_spreed_sheet['OEM']=='Renault'].to_csv('test.csv')

In [ ]:
px.scatter(df_spreed_sheet[df_spreed_sheet['OEM']=='Renault'], x='odometer', y='soh', trendline='ols')

In [ ]:
for oem, trend in trendine_dict.items():
    print(trendine_dict[oem])
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_spreed_sheet[df_spreed_sheet['OEM']==oem]['odometer'],
        y=df_spreed_sheet[df_spreed_sheet['OEM']==oem]['soh'],
        mode='markers',
        marker_color='rgba(50, 182, 193, .9)',
        name='SoH compute'
    ))

    fig.add_trace(go.Scatter(
        x=np.linspace(100,200000, 1000),
        y=trendline_apply(np.linspace(100, 200000, 1000), trendine_dict[oem][0]['trendline']),
        mode='lines',
        line=dict(color='red'),
        name='Fit'
    ))
    
    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title='Odometer',
        yaxis_title='State of Health (SoH)',
        legend_title='Légende',
        title=f"Marque: {oem}",
        template='plotly_white'
    )

    fig.show()

In [ ]:
for oem, trend in trendine_dict.items():
    print(trendine_dict[oem])
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_spreed_sheet[df_spreed_sheet['OEM']==oem]['odometer'],
        y=df_spreed_sheet[df_spreed_sheet['OEM']==oem]['soh'],
        mode='markers',
        marker_color='rgba(50, 182, 193, .9)',
        name='SoH compute'
    ))

    fig.add_trace(go.Scatter(
        x=np.linspace(100,200000, 1000),
        y=trendline_apply(np.linspace(100, 200000, 1000), trendine_dict[oem][0]['trendline']),
        mode='lines',
        line=dict(color='red'),
        name='Fit'
    ))
    
    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title='Odometer',
        yaxis_title='State of Health (SoH)',
        legend_title='Légende',
        title=f"Marque: {oem}",
        template='plotly_white'
    )

    fig.show()